<a href="https://colab.research.google.com/github/smiling621/ML-practice/blob/main/Generative_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generative Network-->is a type of neural network which generates new data samples like our given datasets.
All Classifications algos(SVM,LOGISTIC,REGRESSION,DT,KNN).Neural network is unsupervised and classification method cuz its not labelled.all classification algo are discriminative.two main parts:generator and discriminator.deep learning huda huda kina generator chaeyo vnda dl ley data herera classify matra garxa naya kura generate garna skdaena.Generator-->concept of generating and discriminator-->came from discriminative.Discriminative is always binary kinaki generate vako data original data snga milxa ki mildaena vanera herxa.SVM,LOGISTIC,REGRESSION,DT,KNN are discriminative in nature since they would classify the classes in discriminative manner.
Unlike, discriminative algos(which looks the input data and classifies it),Generative networks learns the underlying distribution of data and produces the new data.

Types of GAN
Deep learning-->PCA(principle component analysis ley xariyeko data lae euta milne plane ma xanera rakhxa)-->linear dimensionality reducer-->Autoencoder(data lae compress garne latin ma puge paxi decode garxa)-->variational autoencoder(VAE)-->Vector Quantized Variational auto encoder(VQVAE),Generative network,VQGAN

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

In [ ]:
def build_generator():
  model=tf.keras.Sequential([
      layers.Dense(256,activation='relu',input_dim=100),
      layers.BatchNormalization(),
      layers.Dense(512,activation='relu'),
      layers.BatchNormalization(),
      layers.Dense(28*28*1,activation='tanh'),
      layers.Reshape((28,28,1))
  ])
  return model

In [ ]:
def build_discriminator():
  model=tf.keras.Sequential([
      layers.Flatten(input_shape=(28,28,1)),
      layers.Dense(512,activation='relu'),
      layers.Dense(256,activation='relu'),
      layers.Dense(1,activation='sigmoid')
  ])
  return model

In [ ]:
(X_train,_),(_,_)=mnist.load_data()

In [ ]:
X_train = (X_train.astype('float32') / 127.5) - 1
X_train = np.expand_dims(X_train, axis=-1)


In [ ]:
X_train

array([[[[[-1.]],

         [[-1.]],

         [[-1.]],

         ...,

         [[-1.]],

         [[-1.]],

         [[-1.]]],


        [[[-1.]],

         [[-1.]],

         [[-1.]],

         ...,

         [[-1.]],

         [[-1.]],

         [[-1.]]],


        [[[-1.]],

         [[-1.]],

         [[-1.]],

         ...,

         [[-1.]],

         [[-1.]],

         [[-1.]]],


        ...,


        [[[-1.]],

         [[-1.]],

         [[-1.]],

         ...,

         [[-1.]],

         [[-1.]],

         [[-1.]]],


        [[[-1.]],

         [[-1.]],

         [[-1.]],

         ...,

         [[-1.]],

         [[-1.]],

         [[-1.]]],


        [[[-1.]],

         [[-1.]],

         [[-1.]],

         ...,

         [[-1.]],

         [[-1.]],

         [[-1.]]]],



       [[[[-1.]],

         [[-1.]],

         [[-1.]],

         ...,

         [[-1.]],

         [[-1.]],

         [[-1.]]],


        [[[-1.]],

         [[-1.]],

         [[-1.]],

         

In [ ]:
discriminator=build_discriminator()
discriminator.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
generator=build_generator()
#now creating the gan by combining the generator and the discriminator
discriminator.trainable=False
gan_input=layers.Input(shape=(100,))
generated_image=generator(gan_input)
gan_output=discriminator(generated_image)
gan = tf.keras.Model(gan_input,gan_output)
gan.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
def plot_and_save_generated_images(epoch, generator, examples = 10, dim = (1,10), figsize = (10, 1), save_dir = 'generated_images'):
  noise = np.random.normal(0, 1, (examples, 100))
  generated_images = generator.predict(noise)
  generated_images = 0.5 * generated_images + 0.5 # rescaling to [0,1]

  plt.figure(figsize = figsize)
  for i in range(examples):
    plt.subplot(dim[0], dim[1], i + 1)
    plt.imshow(generated_images[i, :, :, 0], interpolation = 'nearest', cmap = plt.cm.gray)
    plt.axis('off')
  plt.tight_layout()

  import os
  if not os.path.exists(save_dir):
    os.makedirs(save_dir)
  plt.savefig(f'{save_dir}/gen_generated_on_epoch_{epoch}.png')
  plt.close()

In [ ]:
def train_gan(epochs = 1000, batch_size = 128, save_interval = 100):
  for epoch in range(epochs):
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_images = X_train[idx]
    noise = np.random.normal(0, 1, (batch_size, 100))
    fake_images = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, 100))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    if epoch % save_interval == 0:
      print(f'Epoch{epoch} : D Loss : {d_loss}, G Loss : {g_loss}')
      plot_and_save_generated_images(epoch, generator)

train_gan()

4/4 [==============================] - 0s 8ms/step
Epoch0 : D Loss : [0.04439357 1.        ], G Loss : [0.5072136521339417, 0.7890625]
4/4 [==============================] - 0s 4ms/step
Epoch100 : D Loss : [2.14582751e-05 1.00000000e+00], G Loss : [7.088698387145996, 0.0078125]
4/4 [==============================] - 0s 4ms/step
Epoch200 : D Loss : [0.03551807 0.99609375], G Loss : [6.092354774475098, 0.015625]
4/4 [==============================] - 0s 5ms/step
Epoch300 : D Loss : [0.20919735 0.91015625], G Loss : [4.22378396987915, 0.03125]
4/4 [==============================] - 0s 6ms/step
Epoch400 : D Loss : [0.03598928 1.        ], G Loss : [5.252248764038086, 0.0078125]
4/4 [==============================] - 0s 5ms/step
Epoch500 : D Loss : [0.1079305 0.984375 ], G Loss : [4.46723747253418, 0.0]
4/4 [==============================] - 0s 7ms/step
Epoch600 : D Loss : [0.57812452 0.7578125 ], G Loss : [4.823652267456055, 0.015625]
4/4 [==============================] - 0s 5ms/step
Epoc